**(C)** [Maanav Bindra](https://github.com/maanavb), 2021 ([maanavbindra@gmail.com](mailto:maanavbindra@gmail.com)), all rights reserved.

# DIY Covid-19 Dashboard

This is a DIY Covid-19 Dashboard. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards.

The graph displays a comparison of Covid-19 hospital admissions with ventilator bed occupancy in England (not the United Kingdom). The data is polled from [Public Health England](https://www.gov.uk/government/organisations/public-health-england) API. 

Due to the larger difference in data between the displayed statistics it is possible to view them linear, as well as logarithmic. The graph initially only display data from 2nd of April 2020 till 1st of July 2021 (ventilator bed occupancy was not logged before 2nd of April 2020). 

You may click the button below the graph to refresh the data by polling the PHE API, you have to click on one of the statistics after refreshing to display the new data.

Note: If the API call fails it may be due to connectivity problems or the data might not be available for some reason.


In [ ]:
# importing the libraries needed 
from IPython.display import clear_output
import ipywidgets as wdg 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [ ]:
# "magic" backend provided for iPython, draws static images to the notebook
%matplotlib inline
# increasing the size of the figures
plt.rcParams['figure.dpi'] = 100

In [1]:
# Load JSON files and store the raw data in some variable. This data will initially be displayed on the dashboard when Voila runs.
jsondata={'data': [{'date': '2021-07-01',
   'hospital': 1560,
   'ventilator bed occupancy': 279},  
  {'date': '2021-06-30', 'hospital': 1500, 'ventilator bed occupancy': 264},
  {'date': '2021-06-29', 'hospital': 1445, 'ventilator bed occupancy': 259},
  {'date': '2021-06-28', 'hospital': 1465, 'ventilator bed occupancy': 272},
  {'date': '2021-06-27', 'hospital': 1331, 'ventilator bed occupancy': 244},
  {'date': '2021-06-26', 'hospital': 1276, 'ventilator bed occupancy': 236},
  {'date': '2021-06-25', 'hospital': 1284, 'ventilator bed occupancy': 238},
  {'date': '2021-06-24', 'hospital': 1274, 'ventilator bed occupancy': 240},
  {'date': '2021-06-23', 'hospital': 1255, 'ventilator bed occupancy': 227},
  {'date': '2021-06-22', 'hospital': 1301, 'ventilator bed occupancy': 229},
  {'date': '2021-06-21', 'hospital': 1290, 'ventilator bed occupancy': 212},
  {'date': '2021-06-20', 'hospital': 1168, 'ventilator bed occupancy': 196},
  {'date': '2021-06-19', 'hospital': 1143, 'ventilator bed occupancy': 203},
  {'date': '2021-06-18', 'hospital': 1170, 'ventilator bed occupancy': 210},
  {'date': '2021-06-17', 'hospital': 1122, 'ventilator bed occupancy': 197},
  {'date': '2021-06-16', 'hospital': 1057, 'ventilator bed occupancy': 192},
  {'date': '2021-06-15', 'hospital': 1030, 'ventilator bed occupancy': 187},
  {'date': '2021-06-14', 'hospital': 993, 'ventilator bed occupancy': 170},
  {'date': '2021-06-13', 'hospital': 947, 'ventilator bed occupancy': 171},
  {'date': '2021-06-12', 'hospital': 915, 'ventilator bed occupancy': 152},
  {'date': '2021-06-11', 'hospital': 884, 'ventilator bed occupancy': 146},
  {'date': '2021-06-10', 'hospital': 906, 'ventilator bed occupancy': 140},
  {'date': '2021-06-09', 'hospital': 876, 'ventilator bed occupancy': 141},
  {'date': '2021-06-08', 'hospital': 879, 'ventilator bed occupancy': 140},
  {'date': '2021-06-07', 'hospital': 860, 'ventilator bed occupancy': 133},
  {'date': '2021-06-06', 'hospital': 807, 'ventilator bed occupancy': 131},
  {'date': '2021-06-05', 'hospital': 782, 'ventilator bed occupancy': 124},
  {'date': '2021-06-04', 'hospital': 805, 'ventilator bed occupancy': 119},
  {'date': '2021-06-03', 'hospital': 779, 'ventilator bed occupancy': 124},
  {'date': '2021-06-02', 'hospital': 801, 'ventilator bed occupancy': 116},
  {'date': '2021-06-01', 'hospital': 776, 'ventilator bed occupancy': 123},
  {'date': '2021-05-31', 'hospital': 773, 'ventilator bed occupancy': 110},
  {'date': '2021-05-30', 'hospital': 755, 'ventilator bed occupancy': 115},
  {'date': '2021-05-29', 'hospital': 748, 'ventilator bed occupancy': 112},
  {'date': '2021-05-28', 'hospital': 743, 'ventilator bed occupancy': 116},
  {'date': '2021-05-27', 'hospital': 742, 'ventilator bed occupancy': 110},
  {'date': '2021-05-26', 'hospital': 745, 'ventilator bed occupancy': 115},
  {'date': '2021-05-25', 'hospital': 765, 'ventilator bed occupancy': 117},
  {'date': '2021-05-24', 'hospital': 797, 'ventilator bed occupancy': 113},
  {'date': '2021-05-23', 'hospital': 756, 'ventilator bed occupancy': 111},
  {'date': '2021-05-22', 'hospital': 730, 'ventilator bed occupancy': 113},
  {'date': '2021-05-21', 'hospital': 766, 'ventilator bed occupancy': 117},
  {'date': '2021-05-20', 'hospital': 749, 'ventilator bed occupancy': 115},
  {'date': '2021-05-19', 'hospital': 757, 'ventilator bed occupancy': 113},
  {'date': '2021-05-18', 'hospital': 749, 'ventilator bed occupancy': 114},
  {'date': '2021-05-17', 'hospital': 798, 'ventilator bed occupancy': 117},
  {'date': '2021-05-16', 'hospital': 801, 'ventilator bed occupancy': 116},
  {'date': '2021-05-15', 'hospital': 810, 'ventilator bed occupancy': 115},
  {'date': '2021-05-14', 'hospital': 818, 'ventilator bed occupancy': 115},
  {'date': '2021-05-13', 'hospital': 845, 'ventilator bed occupancy': 119},
  {'date': '2021-05-12', 'hospital': 907, 'ventilator bed occupancy': 123},
  {'date': '2021-05-11', 'hospital': 921, 'ventilator bed occupancy': 126},
  {'date': '2021-05-10', 'hospital': 944, 'ventilator bed occupancy': 135},
  {'date': '2021-05-09', 'hospital': 933, 'ventilator bed occupancy': 131},
  {'date': '2021-05-08', 'hospital': 948, 'ventilator bed occupancy': 135},
  {'date': '2021-05-07', 'hospital': 973, 'ventilator bed occupancy': 145},
  {'date': '2021-05-06', 'hospital': 976, 'ventilator bed occupancy': 144},
  {'date': '2021-05-05', 'hospital': 1032, 'ventilator bed occupancy': 161},
  {'date': '2021-05-04', 'hospital': 1093, 'ventilator bed occupancy': 162},
  {'date': '2021-05-03', 'hospital': 1096, 'ventilator bed occupancy': 168},
  {'date': '2021-05-02', 'hospital': 1082, 'ventilator bed occupancy': 172},
  {'date': '2021-05-01', 'hospital': 1071, 'ventilator bed occupancy': 167},
  {'date': '2021-04-30', 'hospital': 1161, 'ventilator bed occupancy': 170},
  {'date': '2021-04-29', 'hospital': 1235, 'ventilator bed occupancy': 178},
  {'date': '2021-04-28', 'hospital': 1278, 'ventilator bed occupancy': 188},
  {'date': '2021-04-27', 'hospital': 1310, 'ventilator bed occupancy': 196},
  {'date': '2021-04-26', 'hospital': 1393, 'ventilator bed occupancy': 208},
  {'date': '2021-04-25', 'hospital': 1377, 'ventilator bed occupancy': 211},
  {'date': '2021-04-24', 'hospital': 1401, 'ventilator bed occupancy': 220},
  {'date': '2021-04-23', 'hospital': 1478, 'ventilator bed occupancy': 221},
  {'date': '2021-04-22', 'hospital': 1523, 'ventilator bed occupancy': 237},
  {'date': '2021-04-21', 'hospital': 1609, 'ventilator bed occupancy': 249},
  {'date': '2021-04-20', 'hospital': 1649, 'ventilator bed occupancy': 273},
  {'date': '2021-04-19', 'hospital': 1732, 'ventilator bed occupancy': 291},
  {'date': '2021-04-18', 'hospital': 1691, 'ventilator bed occupancy': 283},
  {'date': '2021-04-17', 'hospital': 1735, 'ventilator bed occupancy': 292},
  {'date': '2021-04-16', 'hospital': 1844, 'ventilator bed occupancy': 298},
  {'date': '2021-04-15', 'hospital': 1885, 'ventilator bed occupancy': 294},
  {'date': '2021-04-14', 'hospital': 1972, 'ventilator bed occupancy': 315},
  {'date': '2021-04-13', 'hospital': 2057, 'ventilator bed occupancy': 333},
  {'date': '2021-04-12', 'hospital': 2108, 'ventilator bed occupancy': 337},
  {'date': '2021-04-11', 'hospital': 2096, 'ventilator bed occupancy': 343},
  {'date': '2021-04-10', 'hospital': 2190, 'ventilator bed occupancy': 342},
  {'date': '2021-04-09', 'hospital': 2321, 'ventilator bed occupancy': 362},
  {'date': '2021-04-08', 'hospital': 2382, 'ventilator bed occupancy': 377},
  {'date': '2021-04-07', 'hospital': 2486, 'ventilator bed occupancy': 393},
  {'date': '2021-04-06', 'hospital': 2588, 'ventilator bed occupancy': 406},
  {'date': '2021-04-05', 'hospital': 2680, 'ventilator bed occupancy': 416},
  {'date': '2021-04-04', 'hospital': 2672, 'ventilator bed occupancy': 422},
  {'date': '2021-04-03', 'hospital': 2677, 'ventilator bed occupancy': 433},
  {'date': '2021-04-02', 'hospital': 2784, 'ventilator bed occupancy': 436},
  {'date': '2021-04-01', 'hospital': 2928, 'ventilator bed occupancy': 470},
  {'date': '2021-03-31', 'hospital': 3084, 'ventilator bed occupancy': 501},
  {'date': '2021-03-30', 'hospital': 3283, 'ventilator bed occupancy': 518},
  {'date': '2021-03-29', 'hospital': 3466, 'ventilator bed occupancy': 532},
  {'date': '2021-03-28', 'hospital': 3438, 'ventilator bed occupancy': 532},
  {'date': '2021-03-27', 'hospital': 3676, 'ventilator bed occupancy': 537},
  {'date': '2021-03-26', 'hospital': 3712, 'ventilator bed occupancy': 559},
  {'date': '2021-03-25', 'hospital': 3763, 'ventilator bed occupancy': 566},
  {'date': '2021-03-24', 'hospital': 4005, 'ventilator bed occupancy': 613},
  {'date': '2021-03-23', 'hospital': 4245, 'ventilator bed occupancy': 647},
  {'date': '2021-03-22', 'hospital': 4501, 'ventilator bed occupancy': 676},
  {'date': '2021-03-21', 'hospital': 4492, 'ventilator bed occupancy': 692},
  {'date': '2021-03-20', 'hospital': 4589, 'ventilator bed occupancy': 710},
  {'date': '2021-03-19', 'hospital': 4841, 'ventilator bed occupancy': 749},
  {'date': '2021-03-18', 'hospital': 5083, 'ventilator bed occupancy': 797},
  {'date': '2021-03-17', 'hospital': 5397, 'ventilator bed occupancy': 846},
  {'date': '2021-03-16', 'hospital': 5664, 'ventilator bed occupancy': 882},
  {'date': '2021-03-15', 'hospital': 5976, 'ventilator bed occupancy': 930},
  {'date': '2021-03-14', 'hospital': 6039, 'ventilator bed occupancy': 963},
  {'date': '2021-03-13', 'hospital': 6072, 'ventilator bed occupancy': 1000},
  {'date': '2021-03-12', 'hospital': 6391, 'ventilator bed occupancy': 1023},
  {'date': '2021-03-11', 'hospital': 6687, 'ventilator bed occupancy': 1101},
  {'date': '2021-03-10', 'hospital': 6975, 'ventilator bed occupancy': 1137},
  {'date': '2021-03-09', 'hospital': 7451, 'ventilator bed occupancy': 1187},
  {'date': '2021-03-08', 'hospital': 7847, 'ventilator bed occupancy': 1236},
  {'date': '2021-03-07', 'hospital': 7812, 'ventilator bed occupancy': 1273},
  {'date': '2021-03-06', 'hospital': 8021, 'ventilator bed occupancy': 1326},
  {'date': '2021-03-05', 'hospital': 8594, 'ventilator bed occupancy': 1417},
  {'date': '2021-03-04', 'hospital': 9092, 'ventilator bed occupancy': 1454},
  {'date': '2021-03-03', 'hospital': 9594, 'ventilator bed occupancy': 1507},
  {'date': '2021-03-02', 'hospital': 10121, 'ventilator bed occupancy': 1556},
  {'date': '2021-03-01', 'hospital': 10765, 'ventilator bed occupancy': 1658},
  {'date': '2021-02-28', 'hospital': 10663, 'ventilator bed occupancy': 1630},
  {'date': '2021-02-27', 'hospital': 11090, 'ventilator bed occupancy': 1747},
  {'date': '2021-02-26', 'hospital': 11781, 'ventilator bed occupancy': 1808},
  {'date': '2021-02-25', 'hospital': 12449, 'ventilator bed occupancy': 1866},
  {'date': '2021-02-24', 'hospital': 13007, 'ventilator bed occupancy': 1931},
  {'date': '2021-02-23', 'hospital': 13511, 'ventilator bed occupancy': 1956},
  {'date': '2021-02-22', 'hospital': 14137, 'ventilator bed occupancy': 2072},
  {'date': '2021-02-21', 'hospital': 14142, 'ventilator bed occupancy': 2122},
  {'date': '2021-02-20', 'hospital': 14316, 'ventilator bed occupancy': 2142},
  {'date': '2021-02-19', 'hospital': 15018, 'ventilator bed occupancy': 2251},
  {'date': '2021-02-18', 'hospital': 15633, 'ventilator bed occupancy': 2316},
  {'date': '2021-02-17', 'hospital': 16458, 'ventilator bed occupancy': 2393},
  {'date': '2021-02-16', 'hospital': 17093, 'ventilator bed occupancy': 2484},
  {'date': '2021-02-15', 'hospital': 17730, 'ventilator bed occupancy': 2577},
  {'date': '2021-02-14', 'hospital': 17787, 'ventilator bed occupancy': 2611},
  {'date': '2021-02-13', 'hospital': 17694, 'ventilator bed occupancy': 2592},
  {'date': '2021-02-12', 'hospital': 19009, 'ventilator bed occupancy': 2688},
  {'date': '2021-02-11', 'hospital': 19983, 'ventilator bed occupancy': 2780},
  {'date': '2021-02-10', 'hospital': 20926, 'ventilator bed occupancy': 2872},
  {'date': '2021-02-09', 'hospital': 22067, 'ventilator bed occupancy': 2911},
  {'date': '2021-02-08', 'hospital': 23020, 'ventilator bed occupancy': 2980},
  {'date': '2021-02-07', 'hospital': 22991, 'ventilator bed occupancy': 3047},
  {'date': '2021-02-06', 'hospital': 23042, 'ventilator bed occupancy': 3099},
  {'date': '2021-02-05', 'hospital': 24402, 'ventilator bed occupancy': 3217},
  {'date': '2021-02-04', 'hospital': 25334, 'ventilator bed occupancy': 3275},
  {'date': '2021-02-03', 'hospital': 26374, 'ventilator bed occupancy': 3328},
  {'date': '2021-02-02', 'hospital': 27397, 'ventilator bed occupancy': 3324},
  {'date': '2021-02-01', 'hospital': 28539, 'ventilator bed occupancy': 3414},
  {'date': '2021-01-31', 'hospital': 28112, 'ventilator bed occupancy': 3366},
  {'date': '2021-01-30', 'hospital': 28571, 'ventilator bed occupancy': 3486},
  {'date': '2021-01-29', 'hospital': 29359, 'ventilator bed occupancy': 3506},
  {'date': '2021-01-28', 'hospital': 30333, 'ventilator bed occupancy': 3585},
  {'date': '2021-01-27', 'hospital': 30846, 'ventilator bed occupancy': 3602},
  {'date': '2021-01-26', 'hospital': 32337, 'ventilator bed occupancy': 3634},
  {'date': '2021-01-25', 'hospital': 32938, 'ventilator bed occupancy': 3694},
  {'date': '2021-01-24', 'hospital': 32907, 'ventilator bed occupancy': 3736},
  {'date': '2021-01-23', 'hospital': 32614, 'ventilator bed occupancy': 3731},
  {'date': '2021-01-22', 'hospital': 33412, 'ventilator bed occupancy': 3727},
  {'date': '2021-01-21', 'hospital': 33235, 'ventilator bed occupancy': 3607},
  {'date': '2021-01-20', 'hospital': 33886, 'ventilator bed occupancy': 3603},
  {'date': '2021-01-19', 'hospital': 34015, 'ventilator bed occupancy': 3603},
  {'date': '2021-01-18', 'hospital': 34336, 'ventilator bed occupancy': 3570},
  {'date': '2021-01-17', 'hospital': 33352, 'ventilator bed occupancy': 3533},
  {'date': '2021-01-16', 'hospital': 32923, 'ventilator bed occupancy': 3525},
  {'date': '2021-01-15', 'hospital': 33362, 'ventilator bed occupancy': 3464},
  {'date': '2021-01-14', 'hospital': 32925, 'ventilator bed occupancy': 3351},
  {'date': '2021-01-13', 'hospital': 32689, 'ventilator bed occupancy': 3307},
  {'date': '2021-01-12', 'hospital': 32202, 'ventilator bed occupancy': 3175},
  {'date': '2021-01-11', 'hospital': 32070, 'ventilator bed occupancy': 3055},
  {'date': '2021-01-10', 'hospital': 30758, 'ventilator bed occupancy': 2963},
  {'date': '2021-01-09', 'hospital': 29462, 'ventilator bed occupancy': 2860},
  {'date': '2021-01-08', 'hospital': 29346, 'ventilator bed occupancy': 2814},
  {'date': '2021-01-07', 'hospital': 28246, 'ventilator bed occupancy': 2654},
  {'date': '2021-01-06', 'hospital': 27727, 'ventilator bed occupancy': 2550},
  {'date': '2021-01-05', 'hospital': 26467, 'ventilator bed occupancy': 2378},
  {'date': '2021-01-04', 'hospital': 26626, 'ventilator bed occupancy': 2310},
  {'date': '2021-01-03', 'hospital': 24957, 'ventilator bed occupancy': 2181},
  {'date': '2021-01-02', 'hospital': 23557, 'ventilator bed occupancy': 2017},
  {'date': '2021-01-01', 'hospital': 22534, 'ventilator bed occupancy': 1940},
  {'date': '2020-12-31', 'hospital': 22728, 'ventilator bed occupancy': 1899},
  {'date': '2020-12-30', 'hospital': 22713, 'ventilator bed occupancy': 1854},
  {'date': '2020-12-29', 'hospital': 21787, 'ventilator bed occupancy': 1728},
  {'date': '2020-12-28', 'hospital': 20426, 'ventilator bed occupancy': 1641},
  {'date': '2020-12-27', 'hospital': 19277, 'ventilator bed occupancy': 1556},
  {'date': '2020-12-26', 'hospital': 18350, 'ventilator bed occupancy': 1498},
  {'date': '2020-12-25', 'hospital': 17701, 'ventilator bed occupancy': 1437},
  {'date': '2020-12-24', 'hospital': 18227, 'ventilator bed occupancy': 1427},
  {'date': '2020-12-23', 'hospital': 17834, 'ventilator bed occupancy': 1374},
  {'date': '2020-12-22', 'hospital': 18063, 'ventilator bed occupancy': 1339},
  {'date': '2020-12-21', 'hospital': 17709, 'ventilator bed occupancy': 1327},
  {'date': '2020-12-20', 'hospital': 16633, 'ventilator bed occupancy': 1267},
  {'date': '2020-12-19', 'hospital': 16183, 'ventilator bed occupancy': 1239},
  {'date': '2020-12-18', 'hospital': 15866, 'ventilator bed occupancy': 1190},
  {'date': '2020-12-17', 'hospital': 15741, 'ventilator bed occupancy': 1188},
  {'date': '2020-12-16', 'hospital': 15465, 'ventilator bed occupancy': 1163},
  {'date': '2020-12-15', 'hospital': 15031, 'ventilator bed occupancy': 1159},
  {'date': '2020-12-14', 'hospital': 15053, 'ventilator bed occupancy': 1127},
  {'date': '2020-12-13', 'hospital': 14460, 'ventilator bed occupancy': 1123},
  {'date': '2020-12-12', 'hospital': 13927, 'ventilator bed occupancy': 1087},
  {'date': '2020-12-11', 'hospital': 13901, 'ventilator bed occupancy': 1117},
  {'date': '2020-12-10', 'hospital': 13796, 'ventilator bed occupancy': 1110},
  {'date': '2020-12-09', 'hospital': 13467, 'ventilator bed occupancy': 1094},
  {'date': '2020-12-08', 'hospital': 13629, 'ventilator bed occupancy': 1118},
  {'date': '2020-12-07', 'hospital': 13616, 'ventilator bed occupancy': 1109},
  {'date': '2020-12-06', 'hospital': 13189, 'ventilator bed occupancy': 1087},
  {'date': '2020-12-05', 'hospital': 12968, 'ventilator bed occupancy': 1086},
  {'date': '2020-12-04', 'hospital': 12987, 'ventilator bed occupancy': 1113},
  {'date': '2020-12-03', 'hospital': 12896, 'ventilator bed occupancy': 1094},
  {'date': '2020-12-02', 'hospital': 13212, 'ventilator bed occupancy': 1149},
  {'date': '2020-12-01', 'hospital': 13507, 'ventilator bed occupancy': 1182},
  {'date': '2020-11-30', 'hospital': 13756, 'ventilator bed occupancy': 1185},
  {'date': '2020-11-29', 'hospital': 13521, 'ventilator bed occupancy': 1214},
  {'date': '2020-11-28', 'hospital': 13208, 'ventilator bed occupancy': 1204},
  {'date': '2020-11-27', 'hospital': 13754, 'ventilator bed occupancy': 1242},
  {'date': '2020-11-26', 'hospital': 13908, 'ventilator bed occupancy': 1264},
  {'date': '2020-11-25', 'hospital': 14240, 'ventilator bed occupancy': 1300},
  {'date': '2020-11-24', 'hospital': 14506, 'ventilator bed occupancy': 1306},
  {'date': '2020-11-23', 'hospital': 14712, 'ventilator bed occupancy': 1299},
  {'date': '2020-11-22', 'hospital': 14354, 'ventilator bed occupancy': 1259},
  {'date': '2020-11-21', 'hospital': 14118, 'ventilator bed occupancy': 1248},
  {'date': '2020-11-20', 'hospital': 14236, 'ventilator bed occupancy': 1241},
  {'date': '2020-11-19', 'hospital': 14479, 'ventilator bed occupancy': 1241},
  {'date': '2020-11-18', 'hospital': 14490, 'ventilator bed occupancy': 1242},
  {'date': '2020-11-17', 'hospital': 14411, 'ventilator bed occupancy': 1228},
  {'date': '2020-11-16', 'hospital': 14313, 'ventilator bed occupancy': 1198},
  {'date': '2020-11-15', 'hospital': 13886, 'ventilator bed occupancy': 1194},
  {'date': '2020-11-14', 'hospital': 13399, 'ventilator bed occupancy': 1162},
  {'date': '2020-11-13', 'hospital': 13328, 'ventilator bed occupancy': 1158},
  {'date': '2020-11-12', 'hospital': 12967, 'ventilator bed occupancy': 1088},
  {'date': '2020-11-11', 'hospital': 12730, 'ventilator bed occupancy': 1081},
  {'date': '2020-11-10', 'hospital': 12033, 'ventilator bed occupancy': 1010},
  {'date': '2020-11-09', 'hospital': 12259, 'ventilator bed occupancy': 1048},
  {'date': '2020-11-08', 'hospital': 11680, 'ventilator bed occupancy': 1017},
  {'date': '2020-11-07', 'hospital': 11514, 'ventilator bed occupancy': 1003},
  {'date': '2020-11-06', 'hospital': 11181, 'ventilator bed occupancy': 984},
  {'date': '2020-11-05', 'hospital': 10994, 'ventilator bed occupancy': 986},
  {'date': '2020-11-04', 'hospital': 11037, 'ventilator bed occupancy': 997},
  {'date': '2020-11-03', 'hospital': 10971, 'ventilator bed occupancy': 954},
  {'date': '2020-11-02', 'hospital': 10397, 'ventilator bed occupancy': 883},
  {'date': '2020-11-01', 'hospital': 9623, 'ventilator bed occupancy': 804},
  {'date': '2020-10-31', 'hospital': 9782, 'ventilator bed occupancy': 816},
  {'date': '2020-10-30', 'hospital': 9295, 'ventilator bed occupancy': 801},
  {'date': '2020-10-29', 'hospital': 9127, 'ventilator bed occupancy': 803},
  {'date': '2020-10-28', 'hospital': 9070, 'ventilator bed occupancy': 788},
  {'date': '2020-10-27', 'hospital': 8595, 'ventilator bed occupancy': 743},
  {'date': '2020-10-26', 'hospital': 7856, 'ventilator bed occupancy': 682},
  {'date': '2020-10-25', 'hospital': 7612, 'ventilator bed occupancy': 663},
  {'date': '2020-10-24', 'hospital': 7176, 'ventilator bed occupancy': 632},
  {'date': '2020-10-23', 'hospital': 6842, 'ventilator bed occupancy': 602},
  {'date': '2020-10-22', 'hospital': 6345, 'ventilator bed occupancy': 564},
  {'date': '2020-10-21', 'hospital': 6271, 'ventilator bed occupancy': 572},
  {'date': '2020-10-20', 'hospital': 6072, 'ventilator bed occupancy': 560},
  {'date': '2020-10-19', 'hospital': 5644, 'ventilator bed occupancy': 529},
  {'date': '2020-10-18', 'hospital': 5202, 'ventilator bed occupancy': 504},
  {'date': '2020-10-17', 'hospital': 5029, 'ventilator bed occupancy': 495},
  {'date': '2020-10-16', 'hospital': 4854, 'ventilator bed occupancy': 482},
  {'date': '2020-10-15', 'hospital': 4569, 'ventilator bed occupancy': 482},
  {'date': '2020-10-14', 'hospital': 4313, 'ventilator bed occupancy': 468},
  {'date': '2020-10-13', 'hospital': 4105, 'ventilator bed occupancy': 441},
  {'date': '2020-10-12', 'hospital': 3827, 'ventilator bed occupancy': 426},
  {'date': '2020-10-11', 'hospital': 3604, 'ventilator bed occupancy': 401},
  {'date': '2020-10-10', 'hospital': 3369, 'ventilator bed occupancy': 396},
  {'date': '2020-10-09', 'hospital': 3238, 'ventilator bed occupancy': 367},
  {'date': '2020-10-08', 'hospital': 3179, 'ventilator bed occupancy': 368},
  {'date': '2020-10-07', 'hospital': 3066, 'ventilator bed occupancy': 376},
  {'date': '2020-10-06', 'hospital': 2903, 'ventilator bed occupancy': 349},
  {'date': '2020-10-05', 'hospital': 2705, 'ventilator bed occupancy': 331},
  {'date': '2020-10-04', 'hospital': 2435, 'ventilator bed occupancy': 310},
  {'date': '2020-10-03', 'hospital': 2290, 'ventilator bed occupancy': 307},
  {'date': '2020-10-02', 'hospital': 2180, 'ventilator bed occupancy': 311},
  {'date': '2020-10-01', 'hospital': 2069, 'ventilator bed occupancy': 285},
  {'date': '2020-09-30', 'hospital': 2036, 'ventilator bed occupancy': 282},
  {'date': '2020-09-29', 'hospital': 1954, 'ventilator bed occupancy': 259},
  {'date': '2020-09-28', 'hospital': 1955, 'ventilator bed occupancy': 245},
  {'date': '2020-09-27', 'hospital': 1793, 'ventilator bed occupancy': 233},
  {'date': '2020-09-26', 'hospital': 1689, 'ventilator bed occupancy': 223},
  {'date': '2020-09-25', 'hospital': 1686, 'ventilator bed occupancy': 227},
  {'date': '2020-09-24', 'hospital': 1546, 'ventilator bed occupancy': 209},
  {'date': '2020-09-23', 'hospital': 1439, 'ventilator bed occupancy': 192},
  {'date': '2020-09-22', 'hospital': 1378, 'ventilator bed occupancy': 179},
  {'date': '2020-09-21', 'hospital': 1299, 'ventilator bed occupancy': 154},
  {'date': '2020-09-20', 'hospital': 1174, 'ventilator bed occupancy': 142},
  {'date': '2020-09-19', 'hospital': 1081, 'ventilator bed occupancy': 123},
  {'date': '2020-09-18', 'hospital': 1020, 'ventilator bed occupancy': 115},
  {'date': '2020-09-17', 'hospital': 984, 'ventilator bed occupancy': 108},
  {'date': '2020-09-16', 'hospital': 929, 'ventilator bed occupancy': 107},
  {'date': '2020-09-15', 'hospital': 891, 'ventilator bed occupancy': 101},
  {'date': '2020-09-14', 'hospital': 812, 'ventilator bed occupancy': 88},
  {'date': '2020-09-13', 'hospital': 691, 'ventilator bed occupancy': 74},
  {'date': '2020-09-12', 'hospital': 663, 'ventilator bed occupancy': 70},
  {'date': '2020-09-11', 'hospital': 622, 'ventilator bed occupancy': 129},
  {'date': '2020-09-10', 'hospital': 572, 'ventilator bed occupancy': 62},
  {'date': '2020-09-09', 'hospital': 557, 'ventilator bed occupancy': 64},
  {'date': '2020-09-08', 'hospital': 535, 'ventilator bed occupancy': 64},
  {'date': '2020-09-07', 'hospital': 555, 'ventilator bed occupancy': 62},
  {'date': '2020-09-06', 'hospital': 483, 'ventilator bed occupancy': 52},
  {'date': '2020-09-05', 'hospital': 471, 'ventilator bed occupancy': 50},
  {'date': '2020-09-04', 'hospital': 474, 'ventilator bed occupancy': 54},
  {'date': '2020-09-03', 'hospital': 468, 'ventilator bed occupancy': 58},
  {'date': '2020-09-02', 'hospital': 451, 'ventilator bed occupancy': 59},
  {'date': '2020-09-01', 'hospital': 496, 'ventilator bed occupancy': 59},
  {'date': '2020-08-31', 'hospital': 495, 'ventilator bed occupancy': 54},
  {'date': '2020-08-30', 'hospital': 479, 'ventilator bed occupancy': 52},
  {'date': '2020-08-29', 'hospital': 478, 'ventilator bed occupancy': 56},
  {'date': '2020-08-28', 'hospital': 469, 'ventilator bed occupancy': 52},
  {'date': '2020-08-27', 'hospital': 492, 'ventilator bed occupancy': 62},
  {'date': '2020-08-26', 'hospital': 485, 'ventilator bed occupancy': 55},
  {'date': '2020-08-25', 'hospital': 500, 'ventilator bed occupancy': 62},
  {'date': '2020-08-24', 'hospital': 522, 'ventilator bed occupancy': 62},
  {'date': '2020-08-23', 'hospital': 522, 'ventilator bed occupancy': 65},
  {'date': '2020-08-22', 'hospital': 594, 'ventilator bed occupancy': 66},
  {'date': '2020-08-21', 'hospital': 528, 'ventilator bed occupancy': 64},
  {'date': '2020-08-20', 'hospital': 551, 'ventilator bed occupancy': 64},
  {'date': '2020-08-19', 'hospital': 571, 'ventilator bed occupancy': 63},
  {'date': '2020-08-18', 'hospital': 597, 'ventilator bed occupancy': 64},
  {'date': '2020-08-17', 'hospital': 626, 'ventilator bed occupancy': 67},
  {'date': '2020-08-16', 'hospital': 634, 'ventilator bed occupancy': 69},
  {'date': '2020-08-15', 'hospital': 630, 'ventilator bed occupancy': 68},
  {'date': '2020-08-14', 'hospital': 642, 'ventilator bed occupancy': 63},
  {'date': '2020-08-13', 'hospital': 652, 'ventilator bed occupancy': 64},
  {'date': '2020-08-12', 'hospital': 642, 'ventilator bed occupancy': 68},
  {'date': '2020-08-11', 'hospital': 672, 'ventilator bed occupancy': 63},
  {'date': '2020-08-10', 'hospital': 714, 'ventilator bed occupancy': 60},
  {'date': '2020-08-09', 'hospital': 611, 'ventilator bed occupancy': 56},
  {'date': '2020-08-08', 'hospital': 650, 'ventilator bed occupancy': 55},
  {'date': '2020-08-07', 'hospital': 711, 'ventilator bed occupancy': 57},
  {'date': '2020-08-06', 'hospital': 756, 'ventilator bed occupancy': 61},
  {'date': '2020-08-05', 'hospital': 805, 'ventilator bed occupancy': 63},
  {'date': '2020-08-04', 'hospital': 807, 'ventilator bed occupancy': 65},
  {'date': '2020-08-03', 'hospital': 842, 'ventilator bed occupancy': 67},
  {'date': '2020-08-02', 'hospital': 847, 'ventilator bed occupancy': 72},
  {'date': '2020-08-01', 'hospital': 879, 'ventilator bed occupancy': 74},
  {'date': '2020-07-31', 'hospital': 870, 'ventilator bed occupancy': 71},
  {'date': '2020-07-30', 'hospital': 898, 'ventilator bed occupancy': 70},
  {'date': '2020-07-29', 'hospital': 927, 'ventilator bed occupancy': 72},
  {'date': '2020-07-28', 'hospital': 944, 'ventilator bed occupancy': 78},
  {'date': '2020-07-27', 'hospital': 945, 'ventilator bed occupancy': 83},
  {'date': '2020-07-26', 'hospital': 965, 'ventilator bed occupancy': 81},
  {'date': '2020-07-25', 'hospital': 977, 'ventilator bed occupancy': 82},
  {'date': '2020-07-24', 'hospital': 1018, 'ventilator bed occupancy': 92},
  {'date': '2020-07-23', 'hospital': 1095, 'ventilator bed occupancy': 101},
  {'date': '2020-07-22', 'hospital': 1305, 'ventilator bed occupancy': 97},
  {'date': '2020-07-21', 'hospital': 1188, 'ventilator bed occupancy': 106},
  {'date': '2020-07-20', 'hospital': 1278, 'ventilator bed occupancy': 112},
  {'date': '2020-07-19', 'hospital': 1301, 'ventilator bed occupancy': 119},
  {'date': '2020-07-18', 'hospital': 1321, 'ventilator bed occupancy': 118},
  {'date': '2020-07-17', 'hospital': 1346, 'ventilator bed occupancy': 122},
  {'date': '2020-07-16', 'hospital': 1348, 'ventilator bed occupancy': 118},
  {'date': '2020-07-15', 'hospital': 1489, 'ventilator bed occupancy': 136},
  {'date': '2020-07-14', 'hospital': 1506, 'ventilator bed occupancy': 123},
  {'date': '2020-07-13', 'hospital': 1612, 'ventilator bed occupancy': 138},
  {'date': '2020-07-12', 'hospital': 1611, 'ventilator bed occupancy': 139},
  {'date': '2020-07-11', 'hospital': 1640, 'ventilator bed occupancy': 152},
  {'date': '2020-07-10', 'hospital': 1746, 'ventilator bed occupancy': 162},
  {'date': '2020-07-09', 'hospital': 1801, 'ventilator bed occupancy': 167},
  {'date': '2020-07-08', 'hospital': 1888, 'ventilator bed occupancy': 164},
  {'date': '2020-07-07', 'hospital': 2010, 'ventilator bed occupancy': 178},
  {'date': '2020-07-06', 'hospital': 2099, 'ventilator bed occupancy': 188},
  {'date': '2020-07-05', 'hospital': 2100, 'ventilator bed occupancy': 190},
  {'date': '2020-07-04', 'hospital': 2152, 'ventilator bed occupancy': 186},
  {'date': '2020-07-03', 'hospital': 2294, 'ventilator bed occupancy': 202},
  {'date': '2020-07-02', 'hospital': 2410, 'ventilator bed occupancy': 211},
  {'date': '2020-07-01', 'hospital': 2509, 'ventilator bed occupancy': 220},
  {'date': '2020-06-30', 'hospital': 2665, 'ventilator bed occupancy': 238},
  {'date': '2020-06-29', 'hospital': 3024, 'ventilator bed occupancy': 228},
  {'date': '2020-06-28', 'hospital': 2818, 'ventilator bed occupancy': 236},
  {'date': '2020-06-27', 'hospital': 2819, 'ventilator bed occupancy': 233},
  {'date': '2020-06-26', 'hospital': 2859, 'ventilator bed occupancy': 226},
  {'date': '2020-06-25', 'hospital': 3066, 'ventilator bed occupancy': 236},
  {'date': '2020-06-24', 'hospital': 3223, 'ventilator bed occupancy': 264},
  {'date': '2020-06-23', 'hospital': 3288, 'ventilator bed occupancy': 266},
  {'date': '2020-06-22', 'hospital': 3417, 'ventilator bed occupancy': 296},
  {'date': '2020-06-21', 'hospital': 3373, 'ventilator bed occupancy': 265},
  {'date': '2020-06-20', 'hospital': 3423, 'ventilator bed occupancy': 297},
  {'date': '2020-06-19', 'hospital': 3575, 'ventilator bed occupancy': 283},
  {'date': '2020-06-18', 'hospital': 3681, 'ventilator bed occupancy': 299},
  {'date': '2020-06-17', 'hospital': 3826, 'ventilator bed occupancy': 300},
  {'date': '2020-06-16', 'hospital': 3934, 'ventilator bed occupancy': 327},
  {'date': '2020-06-15', 'hospital': 3920, 'ventilator bed occupancy': 327},
  {'date': '2020-06-14', 'hospital': 3973, 'ventilator bed occupancy': 343},
  {'date': '2020-06-13', 'hospital': 4000, 'ventilator bed occupancy': 339},
  {'date': '2020-06-12', 'hospital': 4136, 'ventilator bed occupancy': 336},
  {'date': '2020-06-11', 'hospital': 4179, 'ventilator bed occupancy': 334},
  {'date': '2020-06-10', 'hospital': 4447, 'ventilator bed occupancy': 384},
  {'date': '2020-06-09', 'hospital': 4665, 'ventilator bed occupancy': 427},
  {'date': '2020-06-08', 'hospital': 4830, 'ventilator bed occupancy': 443},
  {'date': '2020-06-07', 'hospital': 4820, 'ventilator bed occupancy': 448},
  {'date': '2020-06-06', 'hospital': 5004, 'ventilator bed occupancy': 473},
  {'date': '2020-06-05', 'hospital': 5213, 'ventilator bed occupancy': 481},
  {'date': '2020-06-04', 'hospital': 5398, 'ventilator bed occupancy': 494},
  {'date': '2020-06-03', 'hospital': 5600, 'ventilator bed occupancy': 523},
  {'date': '2020-06-02', 'hospital': 5827, 'ventilator bed occupancy': 553},
  {'date': '2020-06-01', 'hospital': 5954, 'ventilator bed occupancy': 584},
  {'date': '2020-05-31', 'hospital': 5933, 'ventilator bed occupancy': 606},
  {'date': '2020-05-30', 'hospital': 5972, 'ventilator bed occupancy': 628},
  {'date': '2020-05-29', 'hospital': 6250, 'ventilator bed occupancy': 630},
  {'date': '2020-05-28', 'hospital': 6528, 'ventilator bed occupancy': 663},
  {'date': '2020-05-27', 'hospital': 6836, 'ventilator bed occupancy': 696},
  {'date': '2020-05-26', 'hospital': 7305, 'ventilator bed occupancy': 756},
  {'date': '2020-05-25', 'hospital': 7448, 'ventilator bed occupancy': 771},
  {'date': '2020-05-24', 'hospital': 7415, 'ventilator bed occupancy': 786},
  {'date': '2020-05-23', 'hospital': 7428, 'ventilator bed occupancy': 815},
  {'date': '2020-05-22', 'hospital': 7761, 'ventilator bed occupancy': 852},
  {'date': '2020-05-21', 'hospital': 7903, 'ventilator bed occupancy': 894},
  {'date': '2020-05-20', 'hospital': 8115, 'ventilator bed occupancy': 921},
  {'date': '2020-05-19', 'hospital': 8518, 'ventilator bed occupancy': 976},
  {'date': '2020-05-18', 'hospital': 8668, 'ventilator bed occupancy': 1010},
  {'date': '2020-05-17', 'hospital': 8611, 'ventilator bed occupancy': 1064},
  {'date': '2020-05-16', 'hospital': 8643, 'ventilator bed occupancy': 1101},
  {'date': '2020-05-15', 'hospital': 9079, 'ventilator bed occupancy': 1200},
  {'date': '2020-05-14', 'hospital': 9231, 'ventilator bed occupancy': 1162},
  {'date': '2020-05-13', 'hospital': 9602, 'ventilator bed occupancy': 1198},
  {'date': '2020-05-12', 'hospital': 9791, 'ventilator bed occupancy': 1247},
  {'date': '2020-05-11', 'hospital': 10109, 'ventilator bed occupancy': 1316},
  {'date': '2020-05-10', 'hospital': 9893, 'ventilator bed occupancy': 1362},
  {'date': '2020-05-09', 'hospital': 10176, 'ventilator bed occupancy': 1398},
  {'date': '2020-05-08', 'hospital': 10249, 'ventilator bed occupancy': 1446},
  {'date': '2020-05-07', 'hospital': 10726, 'ventilator bed occupancy': 1493},
  {'date': '2020-05-06', 'hospital': 11227, 'ventilator bed occupancy': 1610},
  {'date': '2020-05-05', 'hospital': 11754, 'ventilator bed occupancy': 1662},
  {'date': '2020-05-04', 'hospital': 12020, 'ventilator bed occupancy': 1724},
  {'date': '2020-05-03', 'hospital': 12119, 'ventilator bed occupancy': 1795},
  {'date': '2020-05-02', 'hospital': 12314, 'ventilator bed occupancy': 1844},
  {'date': '2020-05-01', 'hospital': 12623, 'ventilator bed occupancy': 1954},
  {'date': '2020-04-30', 'hospital': 12922, 'ventilator bed occupancy': 1991},
  {'date': '2020-04-29', 'hospital': 13224, 'ventilator bed occupancy': 2045},
  {'date': '2020-04-28', 'hospital': 13565, 'ventilator bed occupancy': 2093},
  {'date': '2020-04-27', 'hospital': 14255, 'ventilator bed occupancy': 2194},
  {'date': '2020-04-26', 'hospital': 14174, 'ventilator bed occupancy': 2309},
  {'date': '2020-04-25', 'hospital': 14202, 'ventilator bed occupancy': 2405},
  {'date': '2020-04-24', 'hospital': 14642, 'ventilator bed occupancy': 2414},
  {'date': '2020-04-23', 'hospital': 15402, 'ventilator bed occupancy': 2563},
  {'date': '2020-04-22', 'hospital': 16018, 'ventilator bed occupancy': 2646},
  {'date': '2020-04-21', 'hospital': 16517, 'ventilator bed occupancy': 2701},
  {'date': '2020-04-20', 'hospital': 16654, 'ventilator bed occupancy': 2780},
  {'date': '2020-04-19', 'hospital': 16928, 'ventilator bed occupancy': 2820},
  {'date': '2020-04-18', 'hospital': 16850, 'ventilator bed occupancy': 2868},
  {'date': '2020-04-17', 'hospital': 16728, 'ventilator bed occupancy': 2780},
  {'date': '2020-04-16', 'hospital': 17783, 'ventilator bed occupancy': 2838},
  {'date': '2020-04-15', 'hospital': 17934, 'ventilator bed occupancy': 2847},
  {'date': '2020-04-14', 'hospital': 18552, 'ventilator bed occupancy': 2849},
  {'date': '2020-04-13', 'hospital': 18621, 'ventilator bed occupancy': 2872},
  {'date': '2020-04-12', 'hospital': 18974, 'ventilator bed occupancy': 2881},
  {'date': '2020-04-11', 'hospital': 18604, 'ventilator bed occupancy': 2865},
  {'date': '2020-04-10', 'hospital': 18669, 'ventilator bed occupancy': 2820},
  {'date': '2020-04-09', 'hospital': 18423, 'ventilator bed occupancy': 2546},
  {'date': '2020-04-08', 'hospital': 18152, 'ventilator bed occupancy': 2561},
  {'date': '2020-04-07', 'hospital': 17814, 'ventilator bed occupancy': 2472},
  {'date': '2020-04-06', 'hospital': 17154, 'ventilator bed occupancy': 2264},
  {'date': '2020-04-05', 'hospital': 16657, 'ventilator bed occupancy': 2097},
  {'date': '2020-04-04', 'hospital': 15469, 'ventilator bed occupancy': 1950},
  {'date': '2020-04-03', 'hospital': 13635, 'ventilator bed occupancy': 1788},
  {'date': '2020-04-02', 'hospital': 12135, 'ventilator bed occupancy': 1494}]}

In [ ]:
# loading the data from the JSON file previously created
with open("hospitalvbeds.json", "rt") as INFILE:
    data=json.load(INFILE)

In [ ]:
# retriving the data
datalist=data['data']

In [ ]:
# extracting the dates to sort them in a chronological order
dates=[dictionary['date'] for dictionary in datalist]
# the data might be sorted before, but also contain duplicate values etc. - important to get in the mindset of not blindly trusting data
dates.sort()

In [ ]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [ ]:
startdate=parse_date(dates[0]) # starting date - first index 
enddate=parse_date(dates[-1]) # end date - last index

In [ ]:
def wrangle_data(arg):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    # defining the DataFrame
    temp=arg['data']
    index=pd.date_range(startdate, enddate, freq='D') # date range with the data showing frequency per day
    # title of the columns
    df=pd.DataFrame(index=index, columns=['hospital', 'ventilator bed occupancy'])
    for entry in temp: # each entry is a dictionary with date, hospitalizations and ventilator bed occupancy
        date=parse_date(entry['date'])
        for column in ['hospital', 'ventilator bed occupancy']:
            # check that nothing is there yet - just in case some dates are duplicated,
            # maybe with data for different columns in each entry
            if pd.isna(df.loc[date, column]): 
                # replace None with 0 in our data             
                value=float(entry[column]) if entry[column]!=None else 0.0
                # access a specific location in the dataframe, using .loc
                # and put index,column in a single set of [ ]
                df.loc[date, column]=value # assign to dataframe               
    df.fillna(0.0, inplace=True) # fill in any remaining "holes" due to missing dates         
    return df
            
df=wrangle_data(jsondata) 

In [ ]:
# API access code that will be called by the button callback. 
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    england_only = [ # defining a filter, according to the documentation - I chose to display data from England
    'areaType=nation',
    'areaName=England'
    ]    
    hospital_vbeds = { # defining the structure, a comparison of hospital admissions with ventilator bed occupancy, using the PHE metrics
    'date':'date',
    'hospital':'hospitalCases',
    'ventilator bed occupancy':'covidOccupiedMVBeds'
    }
    api = Cov19API(filters=england_only, structure=hospital_vbeds)
    hospitalvbeds=api.get_json()
    return data # return data read from the API 

In [ ]:
series=wdg.SelectMultiple( # widget letting user select multiple values to display
    options=['hospital', 'ventilator bed occupancy'], # options available for selection
    value=['hospital', 'ventilator bed occupancy'], # initial value
    rows=2, # rows of selection box
    description='Statistics:',
    disabled=False
)

scale=wdg.RadioButtons( # buttons to select whether user want to display graphs linear or logarithmic
    options=['linear', 'logarithmic'], # button options
    description='Scale:', # button description
    disabled=False
)

controls=wdg.HBox([series, scale]) # HBox = horizontal, container to arrange the components

# callback function
def hospitalvbeds_graph(gcols, gscale):
    if gscale=='linear': # if linear scale is showing, logarithmic scale must not show
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        df[list(gcols)].plot(logy=logscale)
        plt.show() # graphs won't update if this is missing 
    else:
        # messages printed if user hasn't selected any columns
        print("Click to select data for graph")
        print("(CTRL+Click to select both categories)")
              
# keep calling the hospitalvbeds_graph (gcols=value_of_series, gscale=value_of_scale)
# not calling the function, but passing the name of it to the interactive output
graph=wdg.interactive_output(hospitalvbeds_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)

In [ ]:
# Printout from this function will be lost in Voila unless captured in an output widget - therefore, we give feedback to the user by changing the appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apidata) 
    # the graph won't refresh until the user interacts with the widget.
    button.icon="check" # check sign displayed when button is clicked
    button.disabled=True # disables the button after successful click

    
apibutton=wdg.Button( # button widget from the ipywidgets library
    description='Refresh data', # text that button displays
    style={'description_width': '150px'}, # increasing the size
    layout={'width': '300px'},
    disabled=False,
    button_style='info',
    tooltip='Click to download current Public Health England data', # message when hovering the button
    icon='hospital-user' # icon related to the data displaying
)

# button callback function with the button
apibutton.on_click(api_button_callback) 

display(apibutton)

**Author and Copyright Notice** Made by [Maanav Bindra](https://github.com/maanavb).
*Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*
A big thanks to lecturer [Dr Fabrizio Smeraldi](http://www.eecs.qmul.ac.uk/~fabri/) - Queen Mary University of London.